In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data1=pd.read_csv('bs140513_032310.csv')

In [3]:
data1.head()

,step,customer,age,gender,zipcodeOri,merchant,zipMerchant,category,amount,fraud
0,0,'C1093826151','4','M','28007','M348934600','28007','es_transportation',4.55,0
1,0,'C352968107','2','M','28007','M348934600','28007','es_transportation',39.68,0
2,0,'C2054744914','4','F','28007','M1823072687','28007','es_transportation',26.89,0
3,0,'C1760612790','3','M','28007','M348934600','28007','es_transportation',17.25,0
4,0,'C757503768','5','M','28007','M348934600','28007','es_transportation',35.72,0


In [4]:
#zipcodeOri and zipMerchant doesn't have any predictive power, zero variance(unique value)
data1=data1.drop(columns=['zipcodeOri', 'zipMerchant'])

In [5]:
data1.shape

(594643, 8)

In [6]:
df=data1.groupby('customer').agg({'fraud':'sum','category':'count' }).reset_index().\
rename(columns = {'category':'num_transactions', 'fraud':'num_frauds'})

In [7]:
df['ratio_fraud']=df['num_frauds']/df['num_transactions']

In [8]:
data1=pd.merge(data1, df[['customer', 'ratio_fraud']], on='customer')

In [9]:
data1.head()

,step,customer,age,gender,merchant,category,amount,fraud,ratio_fraud
0,0,'C1093826151','4','M','M348934600','es_transportation',4.55,0,0.0
1,1,'C1093826151','4','M','M855959430','es_hyper',14.38,0,0.0
2,2,'C1093826151','4','M','M348934600','es_transportation',37.21,0,0.0
3,3,'C1093826151','4','M','M348934600','es_transportation',31.63,0,0.0
4,4,'C1093826151','4','M','M348934600','es_transportation',35.86,0,0.0


In [10]:
#dropping customer id
data1=data1.drop(columns=['customer'])

In [11]:
x=list(data1.columns)

In [12]:
x.remove('fraud')

In [13]:
x

['step', 'age', 'gender', 'merchant', 'category', 'amount', 'ratio_fraud']

In [14]:
X=data1.loc[:,x].values
y=data1.iloc[:, 6].values

In [15]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [16]:
labelencoder_X1=LabelEncoder()
X[:,1]=labelencoder_X1.fit_transform(X[:,1])

labelencoder_X2=LabelEncoder()
X[:,2]=labelencoder_X2.fit_transform(X[:,2])

labelencoder_X3=LabelEncoder()
X[:,3]=labelencoder_X3.fit_transform(X[:,3])

labelencoder_X4=LabelEncoder()
X[:,4]=labelencoder_X4.fit_transform(X[:,4])


In [17]:
# Encoding categorical data
onehotencoder = OneHotEncoder(categorical_features=[1,2,3,4])
X = onehotencoder.fit_transform(X).toarray()

In [53]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [18]:
tmp=y[y==1]

In [19]:
X_outlier=X[tmp]

In [20]:
tmp1=y[y==0]

In [21]:
X_normal=X[tmp1]

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X_normal, y[y==0], test_size=0.2, random_state=0)

# Isolation Forest

In [25]:
from sklearn.ensemble import IsolationForest
classifier=IsolationForest(n_estimators=100, random_state=0)
classifier.fit(X_train)

IsolationForest(bootstrap=False, contamination=0.1, max_features=1.0,
        max_samples='auto', n_estimators=100, n_jobs=1, random_state=0,
        verbose=0)

In [27]:
# predictions
y_pred_train = classifier.predict(X_train)
y_pred_test = classifier.predict(X_test)
y_pred_outliers = classifier.predict(X_outlier)

In [29]:
# outliers ----
print("Accuracy:", list(y_pred_outliers).count(-1)/y_pred_outliers.shape[0])


Accuracy: 1.0


In [30]:
scores_pred = classifier.decision_function(X_train)

In [31]:
scores_pred

array([-6.66133815e-16, -6.66133815e-16, -6.66133815e-16, ...,
       -6.66133815e-16, -6.66133815e-16, -6.66133815e-16])

# One Class SVM

In [37]:
#import plotly.plotly as py
#import plotly.graph_objs as go
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm

In [38]:
# fit the model
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)

In [39]:
clf.fit(X_train)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
      max_iter=-1, nu=0.1, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [ ]:
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_pred_outliers = clf.predict(X_outlier)
n_error_train = y_pred_train[y_pred_train == -1].size
n_error_test = y_pred_test[y_pred_test == -1].size
n_error_outliers = y_pred_outliers[y_pred_outliers == 1].size